In [2]:
import geopandas as gpd
import pandas as pd
import unicodedata
import folium
import colombia_data_functions as cdf

low = pd.read_csv('/Users/skoebric/Documents/dGen/agent_outputs_col_low.csv')
mid = pd.read_csv('/Users/skoebric/Documents/dGen/agent_outputs_col_mid.csv')
high = pd.read_csv('/Users/skoebric/Documents/dGen/agent_outputs_col_high.csv')

low['scenario'] = 'low_solar_cost'
mid['scenario'] = 'mid_solar_cost'
high['scenario'] = 'high_solar_cost'

data = pd.concat([low, mid, high], axis=1)

grouped_data = data.groupby(['control_reg_id','control_reg','state','state_id','Departamento','year','scenario'], as_index=False)[['pv_kw_cum']].sum()

state_id_map = dict(zip(data['state'], data['state_id']))

gdf = dept_shape_merger(grouped_data)

ValueError: Grouper for 'control_reg_id' not 1-dimensional

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.colors as mcolors

data = data.loc[data['year'] == 2050]

def hexcolormapper(df, column, colorscale):
    bins = np.percentile(df[column], np.linspace(0,100,15))
    binned = np.digitize(df[column], bins)
    norm = matplotlib.colors.Normalize(vmin=min(binned), vmax=max(binned), clip=False)
    mapper = plt.cm.ScalarMappable(norm=norm, cmap=colorscale)
    df[f'{column}_color'] = np.vectorize(lambda x: mcolors.to_hex(mapper.to_rgba(x)))(binned)
    return df


In [3]:
m = folium.Map(location=[23.63, -102.558], zoom_start=5)

local_gdfs = []
for c in ['Low Cost Scenario','Mid Cost Scenario','High Cost Scenario']:
    local_gdf = hexcolormapper(data, c, 'Greens')
    local_gdf = local_gdf[[f'{c}',f'{c}_color','geometry','state']]
    local_gdf.columns = ['data','color','geometry','state']
    local_gdfs.append(local_gdf)
   
fgs = []
count = 0
for c in ['Low Cost Scenario','Mid Cost Scenario','High Cost Scenario']:
    local_gdf = local_gdfs[count]
    local_gdf = local_gdf.reset_index()

    if count == 0:
        on_switch = True
    else:
        on_switch = False
    count += 1
    
    fg = folium.FeatureGroup(name=c, show=on_switch)
    
    for index, row in local_gdf.iterrows():
        geojson_ = folium.GeoJson(local_gdf.iloc[index:index+1],
               style_function = lambda feature: {
                   'fillColor': feature['properties']['color'],
                   'fillOpacity' : 0.7,
                   'color': '#000000',
                   'weight':0.2
                   })
        popup_ = folium.Popup(
                      f"<b>State:</b> {row['state']}<br>"
                      f"<b>MW Cumulative PV:</b> {int(row['data']) / 1000}<br>"
                      )
        popup_.add_to(geojson_)
        geojson_.add_to(fg)
    fgs.append(fg)
    
for fg in fgs:
    m.add_child(fg)

m.add_child(folium.map.LayerControl(collapsed = False, autoZIndex = True))

m

In [4]:
m.save('naris_mx.html')